# 02 — Cleaning, Alignment & Feature Engineering
This notebook loads the raw price CSVs in `data/raw/` plus the Fear & Greed Index, aligns everything to the shared date window, and engineers per-asset features (returns, moving averages, rolling volatility).

Output: `data/processed/merged_clean.csv` (the dashboard reads this file).

### Load raw inputs
Read all `*_prices.csv` files written by Notebook 01 and the Fear & Greed CSV. The loader supports both the older export format and the current clean format.

In [1]:
import pandas as pd
from pathlib import Path

RAW_DIR = Path("../data/raw")

def load_price_csv(path: str) -> pd.DataFrame:
    # New format (recommended): Date + yfinance columns
    preview = pd.read_csv(path, nrows=5)
    if len(preview.columns) > 0 and str(preview.columns[0]).strip().lower() == "price":
        # Old format: first 3 rows are header/ticker/date artifacts
        df = pd.read_csv(
            path,
            skiprows=3,
            header=None,
            names=["Date", "Close", "High", "Low", "Open", "Volume"],
        )
    else:
        df = pd.read_csv(path)
        if "Date" not in df.columns:
            raise ValueError(f"Expected a 'Date' column in {path}. Got columns: {list(df.columns)}")
        # Keep only the columns we care about (some exports include 'Adj Close')
        keep = [c for c in ["Date", "Open", "High", "Low", "Close", "Volume"] if c in df.columns]
        df = df[keep]

    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"]).set_index("Date").sort_index()
    df.index = df.index.normalize()

    # Ensure numerics
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    return df

def load_fear_greed_csv(path: str) -> pd.DataFrame:
    fg = pd.read_csv(path, parse_dates=["timestamp"])
    fg = fg.rename(columns={"timestamp": "Date"}).set_index("Date").sort_index()
    fg.index = fg.index.normalize()
    fg["FG_Value"] = pd.to_numeric(fg["value"], errors="coerce")
    if "value_classification" not in fg.columns:
        fg["value_classification"] = fg["FG_Value"].apply(
            lambda x: "Extreme Fear" if x < 25 else ("Fear" if x < 45 else ("Neutral" if x < 55 else ("Greed" if x < 75 else "Extreme Greed")))
        )
    return fg

# Load all *_prices.csv saved by 01_data_collection.ipynb
price_files = sorted(RAW_DIR.glob("*_prices.csv"))
if len(price_files) == 0:
    raise FileNotFoundError(f"No *_prices.csv files found in {RAW_DIR.resolve()}. Run 01_data_collection.ipynb first.")

prices = {}
for path in price_files:
    ticker = path.stem.replace("_prices", "").upper()
    prices[ticker] = load_price_csv(str(path))

fg = load_fear_greed_csv(str(RAW_DIR / "fear_greed_index.csv"))

print("Loaded assets:", sorted(prices.keys()))
for t, df_ in prices.items():
    print(f"  {t}: {df_.shape}  ({df_.index.min().date()} → {df_.index.max().date()})")
print(f"  F&G: {fg.shape}  ({fg.index.min().date()} → {fg.index.max().date()})")

Loaded assets: ['BNB', 'BTC', 'ETH']
  BNB: (1079, 5)  (2023-01-02 → 2025-12-15)
  BTC: (1079, 5)  (2023-01-02 → 2025-12-15)
  ETH: (1079, 5)  (2023-01-02 → 2025-12-15)
  F&G: (2876, 4)  (2018-02-01 → 2025-12-20)


### Align to a common date window
Find the overlapping range across all selected assets and the Fear & Greed series so comparisons are apples-to-apples.

In [2]:
mins = [df_.index.min() for df_ in prices.values()] + [fg.index.min()]
maxs = [df_.index.max() for df_ in prices.values()] + [fg.index.max()]

common_start = max(mins)
common_end = min(maxs)

for t in list(prices.keys()):
    prices[t] = prices[t].loc[common_start:common_end]
fg = fg.loc[common_start:common_end]

print("Aligned date window:", common_start, "→", common_end)
print("Aligned shapes:")
for t, df_ in prices.items():
    print(f"  {t}: {df_.shape}")
print(f"  F&G: {fg.shape}")

# Show a sample
first_ticker = sorted(prices.keys())[0]
prices[first_ticker].head()

Aligned date window: 2023-01-02 00:00:00 → 2025-12-15 00:00:00
Aligned shapes:
  BNB: (1079, 5)
  BTC: (1079, 5)
  ETH: (1079, 5)
  F&G: (1078, 4)


,Open,High,Low,Close,Volume
Date,,,,,
2023-01-02,244.152222,247.715271,241.692490,245.535904,357733262
2023-01-03,245.552139,246.831451,244.784042,246.133362,377452410
2023-01-04,246.140900,260.516052,245.704971,259.119690,632924961
2023-01-05,259.076477,259.076477,255.596558,256.422852,342934041
2023-01-06,256.450195,260.221741,255.067703,259.867157,417893264


### Clean duplicates and missing values
Normalize timestamps, remove duplicates, forward-fill small gaps, and ensure numeric columns are ready for feature engineering.

In [3]:
# Drop duplicates (keep first occurrence)
for t in list(prices.keys()):
    prices[t] = prices[t][~prices[t].index.duplicated(keep="first")]
fg = fg[~fg.index.duplicated(keep="first")]

# Forward fill missing values (common for daily financial series)
for t in list(prices.keys()):
    prices[t] = prices[t].ffill()
fg = fg.ffill()

# Final numeric enforcement
for t in list(prices.keys()):
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        if col in prices[t].columns:
            prices[t][col] = pd.to_numeric(prices[t][col], errors="coerce")
fg["FG_Value"] = pd.to_numeric(fg["FG_Value"], errors="coerce")

# Ensure classification exists
if "value_classification" not in fg.columns:
    fg["value_classification"] = fg["FG_Value"].apply(
        lambda x: "Extreme Fear" if x < 25 else ("Fear" if x < 45 else ("Neutral" if x < 55 else ("Greed" if x < 75 else "Extreme Greed")))
    )

print("Duplicates removed and types fixed.")

Duplicates removed and types fixed.


### Feature engineering (per asset)
For each asset, compute daily returns, 7/30-day moving averages, and 30-day rolling volatility.

In [4]:
for t in sorted(prices.keys()):
    df_ = prices[t]

    # Daily Return
    df_[f"{t}_Return"] = df_["Close"].pct_change()

    # Moving Averages
    df_[f"{t}_MA7"]  = df_["Close"].rolling(7).mean()
    df_[f"{t}_MA30"] = df_["Close"].rolling(30).mean()

    # Rolling Volatility (30 days)
    df_[f"{t}_Vol30"] = df_[f"{t}_Return"].rolling(30).std()

    prices[t] = df_
    print(f"{t} features created.")

BNB features created.
BTC features created.
ETH features created.


### Merge assets + sentiment
Inner-join sentiment with each asset’s engineered features on date to produce one tidy analysis table.

In [5]:
# Start from sentiment and inner-join each asset's engineered features
merged = fg[["FG_Value", "value_classification"]].copy()

for t in sorted(prices.keys()):
    df_ = prices[t]
    subset = df_[["Close", "Volume", f"{t}_Return", f"{t}_MA7", f"{t}_MA30", f"{t}_Vol30"]].copy()
    subset = subset.rename(columns={"Close": f"Close_{t}", "Volume": f"Volume_{t}"})
    merged = merged.join(subset, how="inner")

print(f"Merged shape: {merged.shape}")
merged.head()

Merged shape: (1078, 20)


,FG_Value,value_classification,Close_BNB,Volume_BNB,BNB_Return,BNB_MA7,BNB_MA30,BNB_Vol30,Close_BTC,Volume_BTC,BTC_Return,BTC_MA7,BTC_MA30,BTC_Vol30,Close_ETH,Volume_ETH,ETH_Return,ETH_MA7,ETH_MA30,ETH_Vol30
Date,,,,,,,,,,,,,,,,,,,,
2023-01-02,27,Fear,245.535904,357733262,NaN,NaN,NaN,NaN,16688.470703,12097775227,NaN,NaN,NaN,NaN,1214.656616,3765758498,NaN,NaN,NaN,NaN
2023-01-03,26,Fear,246.133362,377452410,0.002433,NaN,NaN,NaN,16679.857422,13903079207,-0.000516,NaN,NaN,NaN,1214.778809,3392972131,0.000101,NaN,NaN,NaN
2023-01-04,29,Fear,259.119690,632924961,0.052761,NaN,NaN,NaN,16863.238281,18421743322,0.010994,NaN,NaN,NaN,1256.526611,6404416893,0.034367,NaN,NaN,NaN
2023-01-05,29,Fear,256.422852,342934041,-0.010408,NaN,NaN,NaN,16836.736328,13692758566,-0.001572,NaN,NaN,NaN,1250.438599,4001786456,-0.004845,NaN,NaN,NaN
2023-01-06,26,Fear,259.867157,417893264,0.013432,NaN,NaN,NaN,16951.968750,14413662913,0.006844,NaN,NaN,NaN,1269.379028,4977252792,0.015147,NaN,NaN,NaN


### Save processed dataset
Drop the initial rows lost to rolling windows (e.g., the first ~30 days) and write the final dataset to `data/processed/merged_clean.csv`.

In [6]:
# Drop NaN rows created by the rolling windows (first ~30 days)
merged_clean = merged.dropna()

# Save to processed folder
merged_clean.to_csv("../data/processed/merged_clean.csv")

print(f"Final cleaned data saved. Shape: {merged_clean.shape}")
print("Columns:", list(merged_clean.columns))
merged_clean.head()

Final cleaned data saved. Shape: (1048, 20)
Columns: ['FG_Value', 'value_classification', 'Close_BNB', 'Volume_BNB', 'BNB_Return', 'BNB_MA7', 'BNB_MA30', 'BNB_Vol30', 'Close_BTC', 'Volume_BTC', 'BTC_Return', 'BTC_MA7', 'BTC_MA30', 'BTC_Vol30', 'Close_ETH', 'Volume_ETH', 'ETH_Return', 'ETH_MA7', 'ETH_MA30', 'ETH_Vol30']


,FG_Value,value_classification,Close_BNB,Volume_BNB,BNB_Return,BNB_MA7,BNB_MA30,BNB_Vol30,Close_BTC,Volume_BTC,BTC_Return,BTC_MA7,BTC_MA30,BTC_Vol30,Close_ETH,Volume_ETH,ETH_Return,ETH_MA7,ETH_MA30,ETH_Vol30
Date,,,,,,,,,,,,,,,,,,,,
2023-02-01,56,Greed,317.472595,616638893,0.016364,310.496102,291.865013,0.022649,23723.769531,26683255504,0.025259,23231.479074,20606.082031,0.023814,1641.792725,8116969489,0.034829,1602.215402,1490.054069,0.028151
2023-02-02,60,Greed,323.349792,909077028,0.018512,313.136566,294.438894,0.022682,23471.871094,32066936882,-0.010618,23294.206752,20832.482487,0.024069,1643.241577,10558081069,0.000882,1607.949062,1504.336161,0.028141
2023-02-03,60,Greed,332.268677,861598551,0.027583,316.585933,296.877193,0.021454,23449.322266,27083066007,-0.000961,23347.148717,21052.018620,0.024180,1664.745605,8169519805,0.013086,1617.461792,1517.943461,0.027785
2023-02-04,58,Greed,330.617096,440935963,-0.004971,320.067758,299.350335,0.021311,23331.847656,15639298538,-0.005010,23390.114118,21268.522331,0.024251,1667.059204,5843302512,0.001390,1630.979527,1531.830815,0.027695
2023-02-05,58,Greed,327.869354,662969265,-0.008311,321.572492,301.617075,0.021514,22955.666016,19564262605,-0.016123,23273.128348,21468.645573,0.024751,1631.645874,6926696531,-0.021243,1628.906703,1543.906376,0.028253
